## 目的

本文件进行数据的预处理工作

总目标是：加载原始数据，应用一系列标准化的清洗步骤，最后保存成一份干净、可随时用于模型训练的文件

In [1]:
import mne
import scipy.io
import numpy as np
import os

In [ ]:
# 由于ICA独立成分分析适合剔除眼电、肌电尾迹等
# 这也就意味着如果我们应用ICA，那么应该是对于一段比较长的连续数据——包含这些成分的数据
# 即对于Raw数据才适合做这样的处理
# 那么由于数据集中的数据已经是Epochs了，所以不做ICA，以免重要的成分被错误剔除

def preprocess_eeg_file(file_path, save_dir):
	"""
	加载、预处理并保存单个EEG数据文件。
	原始为.mat，处理后epochs数据保存为.fif,  labels数据保存为.npy
	"""
	try:
		print(f"--- 开始处理: {os.path.basename(file_path)} ---")
		# 加载数据
		mat_data = scipy.io.loadmat(file_path)
		eeg_data = mat_data['data']
		labels = mat_data['labels']

		# 转换为MNE Epochs对象
		# “Epochs”指的是从一段连续的、不间断的脑电记录中，根据实验中的特定“事件”截取出来的一个个小的数据片段
		ch_names = [f'Ch{i+1}' for i in range(eeg_data.shape[1])] # eeg_data.shape = (trials, channels, samples)
		sampling_freq = 250 # 采样频率为250Hz
		info = mne.create_info(ch_names=ch_names, sfreq=sampling_freq, ch_types='eeg') #需要提供info对象
		eeg_data_volts = eeg_data * 1e-6 # 单位转换; 转为V为单位，满足EpochsArray()的需要
		epochs = mne.EpochsArray(eeg_data_volts, info, verbose=False) # 将numpy数据封装成MNE EpochsArray

		# 陷波滤波
		# 因为对非Raw数据(我们的是Epochs)不太方便直接notch_filter()，所以迂回一下
		# 滤掉来自电网辐射的工频噪声——来自功率谱密度图PSD的观察
		eeg_data_numpy = epochs.get_data() # 返回 shape (n_epochs, n_channels, n_times)
		#因为notch_filter不能直接对epochs数据做，所以要绕一下
		eeg_data_notched = mne.filter.notch_filter(
				x=eeg_data_numpy,
				Fs=epochs.info['sfreq'], # 传入采样率
				freqs=np.array([50.0]),   # 要滤掉的频率
				verbose=False
		)
		epochs = mne.EpochsArray(eeg_data_notched, info, verbose=False)
		# 陷波滤波在坏段剔除之前，防止剔除时受到工频噪声的干扰

		# 带通滤波 (8-30Hz)， 再大一些3-35Hz，保证采集
		# 关于频率选择
		# 运动想象BCI技术的基础是一种叫做 事件相关去同步/同步 (Event-Related Desynchronization/Synchronization, ERD/ERS) 的大脑活动现象
		# 长期的脑科学研究发现，当人进行运动或想象运动时，其大脑感觉运动皮层区域的两种特定脑电波节律会发生显著的能量变化：
		# 	Mu (µ) 节律：频率范围通常在 8-12 Hz。
		# 	Beta (β) 节律：频率范围通常在 13-30 Hz。
		epochs.filter(l_freq=3.0, h_freq=35.0, picks='eeg', verbose=False) # 3-35，比8-30稍大些，更稳妥


		# ----------------------------------------------------
		# **** 基于振幅的坏段剔除 ****
		# ----------------------------------------------------
		# 定义拒绝标准：对于EEG信号，如果峰峰值超过50微伏(μV)，则认为是坏段
		# 50e-6 表示 50 * 10^-6 V，即 50 μV
		initial_trial_count = len(epochs)
		print(f"原始 trial 数量: {initial_trial_count}")

		reject_criteria = dict(eeg=50e-6) 
		
		# 使用 .drop_bad() 方法自动丢弃不符合标准的Epochs
		epochs.drop_bad(reject=reject_criteria, verbose=True)

		# 为了让后续步骤正确运行，我们需要确保标签也同步更新
		labels_kept = labels[0][epochs.selection] # 注意labels的形状是(1, trials)
		# ----------------------------------------------------

		final_trial_count = len(epochs)
		print(f"坏段剔除后剩余 trial 数量: {final_trial_count} (丢弃了 {initial_trial_count - final_trial_count} 个)")


		#  保存处理后的数据
		base_filename = os.path.basename(file_path).replace('.mat', '')
		# 保存Epochs数据
		epoch_save_path = os.path.join(save_dir, f"{base_filename}-epo.fif")
		epochs.save(epoch_save_path, overwrite=True, verbose=False)
		
		# 保存标签数据
		label_save_path = os.path.join(save_dir, f"{base_filename}-labels.npy")
		np.save(label_save_path, labels_kept)
		
		print(f"处理完成，已保存至: {save_dir}")
			
	except Exception as e:
		print(f"!!!!!! 处理文件 {os.path.basename(file_path)} 时发生错误: {e} !!!!!!")


In [6]:
# --- 主逻辑：循环处理所有文件 ---
source_dir = './data/val/'
processed_dir = './data/val-processed/'
os.makedirs(processed_dir, exist_ok=True)

# 获取所有待处理的文件列表
all_files = [f for f in os.listdir(source_dir) if f.endswith('.mat')] # 因为mac会在下面生成.DS_store!!! orz

for filename in all_files:
	full_file_path = os.path.join(source_dir, filename)
	preprocess_eeg_file(full_file_path, processed_dir)

print("\n\n所有文件批量预处理完成！")

--- 开始处理: sub-021_ses-01_task_motorimagery_eeg.mat ---


C:\Users\16155\AppData\Local\Temp\ipykernel_22080\3959141652.py:31: RuntimeWarning: filter_length (1651) is longer than the signal (1000), distortion is likely. Reduce filter length or filter a longer signal.
  eeg_data_notched = mne.filter.notch_filter(


原始 trial 数量: 94
    Rejecting  epoch based on EEG : ['Ch32']
    Rejecting  epoch based on EEG : ['Ch22']
    Rejecting  epoch based on EEG : ['Ch22']
    Rejecting  epoch based on EEG : ['Ch22']
    Rejecting  epoch based on EEG : ['Ch22']
    Rejecting  epoch based on EEG : ['Ch22']
    Rejecting  epoch based on EEG : ['Ch22']
    Rejecting  epoch based on EEG : ['Ch22', 'Ch30', 'Ch32']
    Rejecting  epoch based on EEG : ['Ch30', 'Ch32']
    Rejecting  epoch based on EEG : ['Ch21']
    Rejecting  epoch based on EEG : ['Ch32']
    Rejecting  epoch based on EEG : ['Ch30', 'Ch32']
    Rejecting  epoch based on EEG : ['Ch25']
    Rejecting  epoch based on EEG : ['Ch13']
    Rejecting  epoch based on EEG : ['Ch13']
    Rejecting  epoch based on EEG : ['Ch17']
    Rejecting  epoch based on EEG : ['Ch22']
17 bad epochs dropped
坏段剔除后剩余 trial 数量: 77 (丢弃了 17 个)
处理完成，已保存至: ./data/val-processed/
--- 开始处理: sub-021_ses-02_task_motorimagery_eeg.mat ---


C:\Users\16155\AppData\Local\Temp\ipykernel_22080\3959141652.py:31: RuntimeWarning: filter_length (1651) is longer than the signal (1000), distortion is likely. Reduce filter length or filter a longer signal.
  eeg_data_notched = mne.filter.notch_filter(


原始 trial 数量: 99
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['C

...

原始 trial 数量: 94
    Rejecting  epoch based on EEG : ['Ch27']
    Rejecting  epoch based on EEG : ['Ch15', 'Ch26']
    Rejecting  epoch based on EEG : ['Ch15']
    Rejecting  epoch based on EEG : ['Ch15']
    Rejecting  epoch based on EEG : ['Ch15']
    Rejecting  epoch based on EEG : ['Ch15', 'Ch24']
    Rejecting  epoch based on EEG : ['Ch15', 'Ch22']
    Rejecting  epoch based on EEG : ['Ch22', 'Ch24']
    Rejecting  epoch based on EEG : ['Ch24']
    Rejecting  epoch based on EEG : ['Ch24']
    Rejecting  epoch based on EEG : ['Ch24']
    Rejecting  epoch based on EEG : ['Ch24']
    Rejecting  epoch based on EEG : ['Ch15']
    Rejecting  epoch based on EEG : ['Ch15']
    Rejecting  epoch based on EEG : ['Ch15']
    Rejecting  epoch based on EEG : ['Ch15']
    Rejecting  epoch based on EEG : ['Ch22']
    Rejecting  epoch based on EEG : ['Ch15']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch15', 'Ch22', 'Ch29']
    Rejecting  epoch based on EEG : 

C:\Users\16155\AppData\Local\Temp\ipykernel_22080\3959141652.py:31: RuntimeWarning: filter_length (1651) is longer than the signal (1000), distortion is likely. Reduce filter length or filter a longer signal.
  eeg_data_notched = mne.filter.notch_filter(


原始 trial 数量: 88
    Rejecting  epoch based on EEG : ['Ch29', 'Ch32']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch29']
    Rejecting  epoch based on EEG : ['Ch25']
    Rejecting  epoch based on EEG : ['Ch25']
    Rejecting  epoch based on EEG : ['Ch22', 'Ch25', 'Ch27', 'Ch29']
    Rejecting  epoch based on EEG : ['Ch25']
    Rejecting  epoch based on EEG : ['Ch22', 'Ch23', 'Ch25', 'Ch27', 'Ch29']
    Rejecting  epoch based on EEG : ['Ch22', 'Ch25']
    Rejecting  epoch based on EEG : ['Ch25']
    Rejecting  epoch based on EEG : ['Ch18']
    Rejecting  epoch based on EEG : ['Ch18', 'Ch27']
    Rejecting  epoch based on EEG : ['Ch18']
    Rejecting  epoch based on EEG : ['Ch16', 'Ch18', 'Ch22', 'Ch27']
    Rejecting  epoch based on EEG : ['Ch18']
    Rejecting  epoch based on EEG : ['Ch18', 'Ch27']
    Rejecting  epoch based on EEG : ['Ch16', 'Ch18', 'Ch23', 'Ch27', 'Ch29']
    Rejecting  epoch based on EEG : ['Ch7', 'Ch16', 'Ch18', 'Ch22', 'Ch27']

C:\Users\16155\AppData\Local\Temp\ipykernel_22080\3959141652.py:31: RuntimeWarning: filter_length (1651) is longer than the signal (1000), distortion is likely. Reduce filter length or filter a longer signal.
  eeg_data_notched = mne.filter.notch_filter(


原始 trial 数量: 100
0 bad epochs dropped
坏段剔除后剩余 trial 数量: 100 (丢弃了 0 个)
处理完成，已保存至: ./data/val-processed/


所有文件批量预处理完成！
